In [154]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import os

%matplotlib notebook

In [7]:
def createCircleMask(height, width, rad):
    # rad: ratio from half of height
    circle = np.zeros((height,width))
    skip_circle = cv2.circle(
        circle,(int(width/2),int(height/2)),
        int(height*.5*rad),1,thickness=-1)
    return circle

In [2]:

## 读取图像，解决imread不能读取中文路径的问题
def cv_imread(filePath):
  try:
    cv_img=cv2.imdecode(np.fromfile(filePath,dtype=np.uint8),-1)
    ## imdecode读取的是rgb，如果后续需要opencv处理的话，需要转换成bgr，转换后图片颜色会变化
    ##cv_img=cv2.cvtColor(cv_img,cv2.COLOR_RGB2BGR)
    return cv_img
  except:
    traceback.print_exc()
    return None

In [510]:
MAX_FEATURES = 1000
GOOD_MATCH_PERCENT = .1 #0.15
RANSAC_TH=3

def alignImagesPerspective(im1, im2, mask):
    # Convert images to grayscale
    #im1Gray = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
    #im2Gray = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)
    im1Gray = im1
    im2Gray = im2

    # Detect ORB features and compute descriptors.
    orb = cv2.ORB_create(MAX_FEATURES)
    keypoints1, descriptors1 = orb.detectAndCompute(im1Gray, mask)
    keypoints2, descriptors2 = orb.detectAndCompute(im2Gray, mask)

    # Match features.
    matcher = cv2.DescriptorMatcher_create(cv2.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING)
    #matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = matcher.match(descriptors1, descriptors2, None)

    # Sort matches by score
    matches.sort(key=lambda x: x.distance, reverse=False)

    # Remove not so good matches
    numGoodMatches = int(len(matches) * GOOD_MATCH_PERCENT)
    matches = matches[:numGoodMatches]

    # Draw top matches
    imMatches = cv2.drawMatches(im1, keypoints1, im2, keypoints2, matches, None)
    cv2.imshow("matches.jpg", imMatches)

    # Extract location of good matches
    points1 = np.zeros((len(matches), 2), dtype=np.float32)
    points2 = np.zeros((len(matches), 2), dtype=np.float32)

    for i, match in enumerate(matches):
        points1[i, :] = keypoints1[match.queryIdx].pt
        points2[i, :] = keypoints2[match.trainIdx].pt

    points1 = np.expand_dims(points1, 0)
    points2 = np.expand_dims(points2, 0)

    # Find homography
    h, mask = cv2.findHomography(points1, points2, cv2.RANSAC, RANSAC_TH)
    assert h is not None
    
    print ('alignImagesPerspective')
    print (h)

    # Use homography
    height, width = im2.shape
    im1Reg = cv2.warpPerspective(im1, h, (width, height))

    return im1Reg, h

# !!!!!!!!!!!!!
def alignImagesPerspectiveMulti(im1, im2, mask):
    h = np.identity(3)
    imcur = im1
    sz = im1.shape
    maskcur = mask
    for i in range(15):
        _, h1 = alignImagesPerspective(imcur, im2, mask)
        if np.all(np.abs(h1 - np.identity(3)) < .05):
            break
        h = h1 @ h        
        imcur = cv2.warpPerspective(im1,h,(sz[1],sz[0]))
        maskcur = cv2.warpPerspective(mask,h,(sz[1],sz[0]))
        
        cv2.imshow('mask', maskcur)
        cv2.imshow('out', np.hstack((im2,im1,imcur,np.abs(imcur-im2)))/255.)
        cv2.waitKey(0)
    return imcur, h

In [123]:
def alignImagesAffine(im1, im2, mask):
    im1 = im1 * mask
    im2 = im2 * mask
    h = cv2.estimateRigidTransform(im2, im1, False)
    assert h is not None
    h = np.vstack((h, [[0,0,1]]))

    print ('alignImagesAffine:')
    print (h)

    # Use homography
    height, width = im2.shape
    im1Reg = cv2.warpPerspective(im1, h, (width, height))

    return im1Reg, h

In [566]:
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
DISPLAY=True
RANSAC_TH = 5
def getFlow(im_transformed1, im1):
    '''
    output: 
        flow: size [height, width, 2] 
        flow[:,:,0] is x offset
        flow[:,:,1] is y offset
    '''
    # https://docs.opencv.org/3.4/d4/dee/tutorial_optical_flow.html
    flow = cv2.calcOpticalFlowFarneback(im_transformed1, im1 ,None, 
                                        pyr_scale=0.5, 
                                        levels=3, 
                                        winsize=15,  # 30 good
                                        iterations=3, 
                                        poly_n=5, 
                                        poly_sigma=1.1, flags=0)
    #                                   None, 0.5, 5, 15, 3, 5, 1.5, 0)
    #                                   None, 0.5, 7, 15, 20, 5, 1.1, 0
    return flow
def getH(flow, mask):
    sz = mask.shape
    flow[...,0] *= mask
    flow[...,1] *= mask
    # Image registration 
    # Ref: CS4243 Computer Vision and Pattern Recognition, NUS - Leow Wee Kheng
    select = np.where((np.abs(flow[...,0]) + np.abs(flow[...,1])).flatten()>1)

    i, j = np.meshgrid(range(sz[0]), range(sz[1]), indexing='ij')
    i1 = i+flow[...,1]
    j1 = j+flow[...,0]
    i = i.flatten()[select]
    j = j.flatten()[select]
    i1 = i1.flatten()[select]
    j1 = j1.flatten()[select]
    M = np.vstack([j,i, np.ones_like(j)]).T
    print (M.shape)
    bj = j1
    bi = i1
    aj = np.linalg.inv(M.T @ M) @ M.T @ bj
    ai = np.linalg.inv(M.T @ M) @ M.T @ bi
    H = np.vstack([aj.T, ai.T, [0,0,1]])
    print ('off=\n', H)
    return H

    #coords = np.vstack([xs, ys]).T
    #coords1 = np.vstack([xs1, ys1]).T
    #M = cv2.getAffineTransform(coords, coords1)
    #print (M)

def getH1(flow, mask):
    sz = mask.shape
    flow[...,0] *= mask
    flow[...,1] *= mask
    # Image registration 
    # Ref: CS4243 Computer Vision and Pattern Recognition, NUS - Leow Wee Kheng
    select = np.where((np.abs(flow[...,0]) + np.abs(flow[...,1])).flatten()>0.1)
    if (len(select[0])<3):
        return np.identity(3)
    i, j = np.meshgrid(range(sz[0]), range(sz[1]), indexing='ij')
    i1 = i+flow[...,1]
    j1 = j+flow[...,0]
    i = i.flatten()[select]
    j = j.flatten()[select]
    i1 = i1.flatten()[select]
    j1 = j1.flatten()[select]
    H, mask = cv2.findHomography(np.vstack([j,i]).T, np.vstack([j1,i1]).T,
                                 cv2.RANSAC, RANSAC_TH)
    # !!! Remove projective transform
    H[2,:2]*=0
    print (H)
    return H
    
def showFlow(im_ori, im_transformed, flow, mask):
    flow[...,0] *= mask
    flow[...,1] *= mask
    
    sz = im_ori.shape
    im_out = np.zeros(sz, dtype=np.float32)
    print (im_out.shape)
    for i in range(sz[0]):
        for j in range(sz[1]):
            i1 = int(i+flow[i,j,1])
            j1 = int(j+flow[i,j,0])
            if i1>=0 and j1>=0 and i1<sz[0] and j1<sz[1]:
                im_out[i1][j1] = im_transformed[i,j]
    #plt.figure()
    #plt.imshow(np.hstack((im_ori, im_out)),  cmap='gray')    
    cv2.imshow('flow map', np.hstack((im_ori, im_out))/255.)
    
def showFlow1(im_ori, im_transformed, flow, mask):
    flow[...,0] *= mask
    flow[...,1] *= mask
    mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
    hsv = np.zeros(im_ori.shape+(3,))
    hsv[...,0] = ang*180/np.pi
    hsv[...,1] = 255
    hsv[...,2] = mag/50 #cv2.normalize(mag,None,0,1,cv2.NORM_MINMAX)
    bgr = cv2.cvtColor(hsv.astype(np.float32),cv2.COLOR_HSV2RGB)
    cv2.imshow('flow', bgr)
    
def alignImagesOpticalFlowOnce(im1, imcur, im2, mask1):
    flow = getFlow(imcur, im2)

    if DISPLAY:
        showFlow1(im1, imcur, flow, mask1)        

    H = getH1(flow, mask1)
    
    return H
    
def alignImagesOpticalFlow(im1, im2, mask):
    mask = np.float32(mask)
    im1 = np.float32(im1)
    im2 = np.float32(im2)
    imcur = im1
    sz = im1.shape
    
    H0 = np.identity(3)
    rads = 16
    
    best_h = H0
    best_err = 9999
    
    for r in range(0,rads):
        # rotate
        T_shift_neg = np.array([
            [1, 0, -sz[1]*.5],
            [0, 1, -sz[0]*.5],
            [0, 0, 1]])
        T_shift = np.array([
            [1, 0, sz[1]*.5],
            [0, 1, sz[0]*.5],
            [0, 0, 1]])
        th = np.pi/(rads/2)
        T_rotate = np.array([
            [np.cos(th), -np.sin(th), 0],
            [np.sin(th), np.cos(th), 0],
            [0, 0, 1]])
        H0 = T_shift @ T_rotate @ T_shift_neg @ H0
        H1 = H0
        #H1 = np.array([[1,0,10], [0,1,0], [0,0,1]], dtype=np.float)
        imcur = cv2.warpAffine(im1,H1[:2,:],(sz[1],sz[0]))
        mask = cv2.warpAffine(mask,H1[:2,:],(sz[1],sz[0]))
        cv2.imshow('mask', mask)

        mask1=mask

        for i in range(3):
            H = alignImagesOpticalFlowOnce(im1, imcur, im2, mask1)
            
            H1 = H @ H1
            
            det = np.linalg.det(H1)
            if DISPLAY:
                print ("H1 Det:", det)
            if det>2 or det<.5:
                break
                
            imcur = cv2.warpAffine(im1,H1[:2,:],(sz[1],sz[0]))
            mask1 = cv2.warpAffine(mask,H1[:2,:],(sz[1],sz[0]))
            if DISPLAY:
                cv2.imshow('mask', mask1)
            
            err = np.mean(abs(imcur-im2)[mask1.astype('bool')])
            if err < best_err:
                print ('best_err = ', best_err)
                best_err = err
                best_h = H1

            #print (np.max(imcur), np.max(im2))
            #print (np.max(np.abs(imcur-im2)))
            #plt.figure()
            #print (im1.shape)
            #plt.imshow(np.hstack((im2,im1,imcur,np.abs(imcur-im2))) ,  cmap='gray')
            #plt.pause(.01)
            if DISPLAY:
                cv2.imshow('out', np.hstack((im2,im1,imcur*mask1,np.abs(imcur-im2)))/255.)
                k = cv2.waitKey(1)
                if k==32:
                    break
    H1 = best_h
    for i in range(15):
        imcur = cv2.warpAffine(im1,H1[:2,:],(sz[1],sz[0]))
        mask1 = cv2.warpAffine(mask,H1[:2,:],(sz[1],sz[0]))
        
        H = alignImagesOpticalFlowOnce(im1, imcur, im2, mask)
        H1 = H @ H1

        print ("H1 Det:", det)

        err = np.mean(abs(imcur-im2)[mask1.astype('bool')])
        if err < best_err:
            print ('best_err = ', best_err)
            best_err = err
            best_h = H1

        #print (np.max(imcur), np.max(im2))
        #print (np.max(np.abs(imcur-im2)))
        #plt.figure()
        #print (im1.shape)
        #plt.imshow(np.hstack((im2,im1,imcur,np.abs(imcur-im2))) ,  cmap='gray')
        #plt.pause(.01)
        if DISPLAY:
            cv2.imshow('out', np.hstack((im2,im1,imcur*mask1,np.abs(imcur-im2)))/255.)
            k = cv2.waitKey(1)
            if k==32:
                break
    
    #return best_h
    return H1

In [567]:

#refFilename = "D:/Images/A70/Image_60057B1/鏡筒_A_IPI_4_20180510_154021893.png"
#imFilename = "D:/Images/A70/Image_60057B1/鏡筒_A_IPI_4_20180510_154029071.png"
#mask_rad=.4
#refFilename = "D:\Images\A70\Image_8419A5/鏡筒_A_IPI_4_20190108_040155115.png"
#imFilename = "D:\Images\A70\Image_8419A5/鏡筒_A_IPI_4_20190108_040159422.png"
#mask_rad=.2

refFilename = "D:\Images\A70\Image_60057B1/鏡筒_A_IPI_4_20180510_154021893.png"
path = "D:\Images\A70\Image_60057B1/"
mask_rad=.4
#refFilename = "D:\Images\A70\Image_8419A5/鏡筒_A_IPI_4_20190108_040155115.png"
#path = "D:\Images\A70\Image_8419A5/"
#mask_rad=.2

mask = 1-createCircleMask(im.shape[0], im.shape[1], mask_rad).astype('uint8')
mask *= createCircleMask(im.shape[0], im.shape[1], .85).astype('uint8')

step = 8

# Read reference image
print("Reading reference image : ", refFilename)
imReference = cv_imread(refFilename)
imReference = imReference[::step, ::step]
cv2.imshow('ref', imReference)

# Create array of array of images.
imagePaths = []
i = 0        
k = 0
for root, subdirs, files in os.walk(path):
    print("Reading images from " + root + " ... ", flush=True)      
    for filename in files:
        fileExt = os.path.splitext(filename)[1]
        if fileExt not in [".jpg", ".jpeg", ".png"]:
            continue

        imagePath = os.path.join(root, filename)
            
        # Read image to be aligned
        print("Reading image to align : ", imFilename);  
        im = cv_imread(imagePath)
        im = im[::step, ::step]
        cv2.imshow('im', im)

        if im is None :
            print("Error:{} not read properly".format(imagePath))
            continue
        
        #Reference *= mask
        #im *= mask
        
        print("Aligning images ...")
        # Registered image will be resotred in imReg. 
        # The estimated homography will be stored in h. 
        
        #imReg, h = alignImagesPerspectiveMulti(im, imReference, mask)
        #imReg, h = alignImagesAffine(imReg, imReference, mask)
        h = alignImagesOpticalFlow(im, imReference, mask)
        sz = imReference.shape
        imReg = cv2.warpAffine(im,h[:2,:],(sz[1],sz[0]))        
        
        
        diff = np.abs(imReg.astype('float')-imReference.astype('float')).astype('uint8')
        cv2.imshow('out', np.hstack([imReference*mask, im, imReg, diff ]))

        # Write aligned image to disk. 
        #outFilename = "aligned.jpg"
        #print("Saving aligned image : ", outFilename); 
        #cv2.imwrite(outFilename, imReg)

        # Print estimated homography
        print("Estimated homography : \n",  h)
        print ('Press key to cont.')
        k = cv2.waitKey(0)

        if k==27:
            break
    if k==27:
        break
print ('Done')

Reading reference image :  D:\Images\A70\Image_60057B1/鏡筒_A_IPI_4_20180510_154021893.png
Reading images from D:\Images\A70\Image_60057B1/ ... 
Reading image to align :  D:\Images\A70\Image_8419A5/鏡筒_A_IPI_4_20190108_040159422.png
Aligning images ...
[[ 1.02707202  0.09861668 -7.19927686]
 [-0.06407984  1.04329571  5.90488002]
 [ 0.          0.          1.        ]]
H1 Det: 1.0778591712682881
best_err =  9999
[[ 1.0206221   0.08604332 -6.64215433]
 [-0.08627803  1.01536577  8.85024357]
 [ 0.          0.          1.        ]]
H1 Det: 1.1249922132884014
[[ 0.96726827  0.08970745 -5.3980071 ]
 [-0.1047974   0.98861128 11.2462797 ]
 [ 0.          0.          1.        ]]
H1 Det: 1.086352587139723
[[ 9.86515466e-01  1.24685115e-03 -2.23801958e-01]
 [-2.95396976e-02  9.99103010e-01  2.88334190e+00]
 [-0.00000000e+00  0.00000000e+00  1.00000000e+00]]
H1 Det: 0.9856674028521666
best_err =  24.97377
[[ 1.02392902  0.02480736 -2.65753415]
 [-0.00426204  1.02033531  0.25910726]
 [-0.          0.  

[[ 1.09262704e+00  1.29936258e-01 -1.03900157e+01]
 [-1.41605676e-03  1.11501610e+00 -2.15467453e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]
H1 Det: 1.0768339649868872
[[ 1.03149913  0.00539744 -2.01181905]
 [ 0.03511348  1.00091521 -2.59357107]
 [ 0.         -0.          1.        ]]
H1 Det: 1.032253645411066
[[ 1.0083047  -0.00872105  1.19516495]
 [ 0.01216422  1.03438635 -2.18645297]
 [ 0.          0.          1.        ]]
H1 Det: 1.0767259198730181
[[ 0.93435469 -0.01155651  2.7834714 ]
 [-0.01560425  0.9431686   2.74359577]
 [-0.          0.          1.        ]]
H1 Det: 0.9486748532237919
[[ 1.00806643  0.06804592 -4.26195492]
 [-0.05193347  1.05626389  4.41366887]
 [-0.          0.          1.        ]]
H1 Det: 1.0683180271464485
[[ 0.99905849  0.02552204 -2.88128648]
 [-0.04140653  0.96824537  5.86363827]
 [-0.          0.          1.        ]]
H1 Det: 1.0345490727469144
[[ 0.95551235  0.02634524  0.03907362]
 [-0.04454669  0.98411439  3.81218865]
 [-0.          0.

Reading image to align :  D:\Images\A70\Image_8419A5/鏡筒_A_IPI_4_20190108_040159422.png
Aligning images ...
[[ 1.06517628  0.05415277 -3.67798363]
 [ 0.02525475  1.12788542 -5.34872415]
 [ 0.          0.          1.        ]]
H1 Det: 1.2000291787907498
best_err =  9999
[[ 1.17390343e+00 -3.42942652e-02 -6.96306819e+00]
 [-9.54625426e-04  1.06637548e+00  1.48341786e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]
H1 Det: 1.5021834392372357
[[ 0.77373807  0.09144108 -1.3756263 ]
 [-0.03180277  0.83577538  1.92895764]
 [-0.         -0.          1.        ]]
H1 Det: 0.9757872840250494
best_err =  40.960125
[[ 0.9973761   0.01181472 -1.19192622]
 [-0.02459646  0.9877722   3.23935798]
 [-0.          0.          1.        ]]
H1 Det: 0.9854709834486732
best_err =  22.092766
[[ 0.98885277  0.01014262 -0.31518732]
 [-0.01014916  0.99499892  0.90375484]
 [-0.         -0.          1.        ]]
H1 Det: 0.9697136787205602
[[ 0.99868574  0.00599977 -0.41804696]
 [-0.00481392  0.99919923  0.470

[[ 0.98700753 -0.00666869  0.94018039]
 [-0.00576311  0.98857973  0.76371122]
 [-0.         -0.          1.        ]]
H1 Det: 0.984782827461941
[[ 9.98045141e-01 -9.36413304e-04 -1.13940021e-02]
 [ 2.31944350e-03  9.95275665e-01 -1.27399655e-01]
 [-0.00000000e+00 -0.00000000e+00  1.00000000e+00]]
H1 Det: 0.984782827461941
Estimated homography : 
 [[  0.71234274  -0.66980258  77.11407756]
 [  0.65751055   0.72063185 -45.9854599 ]
 [  0.           0.           1.        ]]
Press key to cont.
Reading image to align :  D:\Images\A70\Image_8419A5/鏡筒_A_IPI_4_20190108_040159422.png
Aligning images ...
[[ 1.00692583e+00  2.97743077e-02 -4.19560551e-01]
 [-9.58065686e-04  1.05448586e+00 -1.83842156e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]
H1 Det: 1.061817568777266
best_err =  9999
[[ 1.02357252 -0.0420443  -0.34096505]
 [-0.02387893  0.97650301  3.44360768]
 [ 0.         -0.          1.        ]]
H1 Det: 1.0602436035308453
best_err =  24.065268
[[ 0.93497753  0.05273655  0.68042

best_err =  9.128062
[[ 9.99737922e-01  8.89177231e-06  1.69742783e-03]
 [ 8.48746520e-05  9.99718289e-01 -5.54520722e-03]
 [-0.00000000e+00  0.00000000e+00  1.00000000e+00]]
H1 Det: 0.9877601633253661
[[ 1.00146121e+00  1.15618124e-04 -6.42474861e-02]
 [ 4.79775805e-04  1.00090460e+00 -4.58389243e-02]
 [-0.00000000e+00  0.00000000e+00  1.00000000e+00]]
H1 Det: 0.9877601633253661
[[ 9.99349502e-01  4.30233315e-04  1.69361227e-02]
 [-3.49603347e-04  1.00020582e+00  8.10149128e-03]
 [-0.00000000e+00  0.00000000e+00  1.00000000e+00]]
H1 Det: 0.9877601633253661
[[ 9.97982196e-01 -3.13590911e-04  8.71639755e-02]
 [-8.02655248e-04  9.98522954e-01  8.81388670e-02]
 [-0.00000000e+00  0.00000000e+00  1.00000000e+00]]
H1 Det: 0.9877601633253661
Estimated homography : 
 [[  0.97884698   0.19257787 -12.57246395]
 [ -0.1949109    0.97562082  21.10534178]
 [  0.           0.           1.        ]]
Press key to cont.
Reading image to align :  D:\Images\A70\Image_8419A5/鏡筒_A_IPI_4_20190108_040159422.p

...

[[ 8.81349437e-01 -5.62795703e-02  9.64894355e+00]
 [-6.19879074e-03  9.31988795e-01  9.48213101e-01]
 [-0.00000000e+00 -0.00000000e+00  1.00000000e+00]]
H1 Det: 0.8210589342703629
[[ 0.96144576 -0.04415943 -0.09077155]
 [ 0.05026162  0.91533007 -3.41758211]
 [-0.         -0.          1.        ]]
H1 Det: 0.7243872402383946
[[ 1.28126685e+00  2.26797617e-02 -8.30922867e+00]
 [ 7.05403086e-03  1.18286043e+00 -1.92508809e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]
H1 Det: 1.0977363267496956
[[ 9.61299198e-01 -4.10803979e-02  2.55400050e+00]
 [ 1.33850046e-03  9.24180453e-01  1.33533821e+00]
 [-0.00000000e+00 -0.00000000e+00  1.00000000e+00]]
H1 Det: 0.8884689139761484
[[ 9.46109115e-01 -3.62488444e-02  2.67665055e+00]
 [-1.88133336e-03  9.39626724e-01  8.83449753e-01]
 [-0.00000000e+00 -0.00000000e+00  1.00000000e+00]]
H1 Det: 0.7897788639000526
[[ 8.60134692e-01 -7.22471738e-02  7.90719280e+00]
 [-6.99029819e-03  9.21516318e-01  1.41443370e-01]
 [-0.00000000e+00 -0.00000000

 [-0.          0.          1.        ]]
H1 Det: 0.9763002534471131
best_err =  15.278589
[[ 0.96192001  0.04038183 -2.02097475]
 [-0.06159578  0.97536124  6.32299931]
 [-0.         -0.          1.        ]]
H1 Det: 0.9407068457841669
[[ 0.99980335  0.07508777 -5.06872907]
 [-0.06494999  1.0202605   5.7528817 ]
 [-0.          0.          1.        ]]
H1 Det: 0.96416508262335
[[ 1.0121865   0.05302446 -4.70364072]
 [-0.06080166  0.99513746  6.67561457]
 [-0.          0.          1.        ]]
H1 Det: 0.9742778975722699
[[ 1.01933382  0.00308021 -0.77510096]
 [ 0.0101063   1.03774472 -2.40451453]
 [ 0.          0.          1.        ]]
H1 Det: 1.0577771515411485
[[ 9.79084632e-01 -4.63441621e-03  1.24947925e+00]
 [-9.93365129e-04  9.84689999e-01  4.08873846e-01]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]
H1 Det: 1.0197926288319128
[[ 0.99373985  0.01300067 -0.1224781 ]
 [-0.00937433  1.01107369  0.50533471]
 [-0.          0.          1.        ]]
H1 Det: 1.0247550365398135
[[ 0.98

 [-0.         -0.          1.        ]]
H1 Det: 0.9740155495378436
[[ 0.99501024  0.0095784  -0.48257374]
 [ 0.0163676   1.00141635 -1.27048069]
 [-0.         -0.          1.        ]]
H1 Det: 0.9962627425366396
[[ 1.01242511 -0.0087847   0.14565614]
 [ 0.00213673  1.0059466  -0.18779854]
 [ 0.         -0.          1.        ]]
H1 Det: 1.0146581074760883
[[ 0.99624474 -0.00494389  0.34442565]
 [ 0.00110865  0.9908915   0.20444855]
 [ 0.         -0.          1.        ]]
H1 Det: 1.001646060231894
[[ 1.00245718  0.01931917 -1.46861662]
 [-0.00718852  1.02002955  0.87938471]
 [-0.          0.          1.        ]]
H1 Det: 1.022674818937427
[[ 1.02639788  0.01598109 -2.34204691]
 [-0.02204461  1.00844182  2.58624155]
 [-0.          0.          1.        ]]
H1 Det: 1.0588926870394155
[[ 1.00900967  0.05615368 -2.30280526]
 [-0.01537676  1.0839588  -0.64682615]
 [-0.          0.          1.        ]]
H1 Det: 1.1590516326818658
[[ 0.98176438 -0.02283273  2.59560541]
 [ 0.02386342  1.00301231 

[[ 1.00074046e+00  1.77783796e-03 -6.92990030e-02]
 [-8.61391876e-04  1.00267975e+00  2.20706206e-02]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]
H1 Det: 1.013545127062311
[[ 9.99677354e-01 -1.31949563e-03  1.02428527e-02]
 [ 4.89653129e-04  9.97158988e-01  3.52761847e-02]
 [ 0.00000000e+00 -0.00000000e+00  1.00000000e+00]]
H1 Det: 1.013545127062311
Estimated homography : 
 [[ -0.75249894  -0.66378184 225.06220245]
 [  0.66205716  -0.75105663  65.49013397]
 [  0.           0.           1.        ]]
Press key to cont.
Reading image to align :  D:\Images\A70\Image_8419A5/鏡筒_A_IPI_4_20190108_040159422.png
Aligning images ...
[[ 0.99371366  0.0365372  -1.35105849]
 [-0.0279022   1.03218015  1.6560057 ]
 [-0.          0.          1.        ]]
H1 Det: 1.0267109866548187
best_err =  9999
[[ 1.02378731  0.02412107 -2.83715483]
 [-0.03217751  1.01805594  3.07805799]
 [ 0.          0.          1.        ]]
H1 Det: 1.0709097719804772
[[ 1.02502     0.03281625 -3.04622294]
 [-0.00825948  1

[[ 9.66874719e-01 -1.89319106e-02  1.85595463e+00]
 [-2.72058683e-04  9.57236312e-01  7.24911645e-01]
 [-0.00000000e+00 -0.00000000e+00  1.00000000e+00]]
H1 Det: 0.9886041204098673
[[ 0.9961679  -0.0295966   0.36307869]
 [-0.00197746  0.9703835   0.79840906]
 [-0.         -0.          1.        ]]
H1 Det: 0.9886041204098673
[[ 1.03317284  0.03294862 -1.80284414]
 [ 0.02578151  1.06146827 -3.98356556]
 [ 0.          0.          1.        ]]
H1 Det: 0.9886041204098673
Estimated homography : 
 [[  0.43437703  -0.9368165  127.03122707]
 [  0.92068261   0.42241613 -47.35618095]
 [  0.           0.           1.        ]]
Press key to cont.
Done


In [525]:
cv2.__version__
#imReg = cv2.warpAffine(im,h[:2,:],(sz[1],sz[0]))        
h

array([[ 9.98469410e-01,  5.82101447e-02, -4.47421788e+00],
       [-6.04712969e-02,  9.90285134e-01,  6.56280098e+00],
       [-6.72630511e-05, -8.17298357e-06,  1.00015356e+00]])